<a href="https://colab.research.google.com/github/Mohivrm/Mohivrm/blob/main/explanable_AI_shap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install shap transformers torch langchain langchain-huggingface langchain-groq lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 8.0 MB/s eta 0:00:00
 

In [2]:
# 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
# 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
# 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
# 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
from langchain_groq import ChatGroq
import getpass
import os

os.environ["GROQ_API_KEY"] = "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"

# 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

In [3]:
#llama3-8b-8192
#mixtral-8x7b-32768
#gemma-7b-it
#llama3-70b-8192
llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0,
    max_tokens=500,  # Limit the number of tokens to avoid explanations
    timeout=None,
    max_retries=2)

llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7fec26e34d50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fec26b0dad0>, model_name='llama3-70b-8192', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=500)

#shap

In [7]:
import shap
import numpy as np
import pandas as pd
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


input_text = """[Patient:
Doctor, my mood has been all over the place. Sometimes, I feel extremely energetic, like I can do anything, and I don’t even need much sleep. My mind races with so many ideas, and I feel unstoppable. But then, out of nowhere, I crash.

During these lows, I feel completely drained, hopeless, and lose interest in everything. Even getting out of bed feels like a struggle. My sleep patterns are all over the place—sometimes I barely sleep, and other times I sleep too much.

I’ve also been making impulsive decisions when I’m feeling high, like spending too much money or talking too fast without thinking. But when the low hits, I regret everything and feel like I’m stuck in a deep hole. I don’t understand why this keeps happening. Could there be something wrong with me?
]"""

# Define the LLM prompt for disease prediction
prompt = PromptTemplate.from_template(
    """
    Predict the disease name from the statement: {question}. Provide only the main disease in this format: **disease name**.

    Guidelines:
    - If no disease is mentioned, return **"No disease detected"**.
    - For multiple diseases, choose the most relevant or primary one.
    - Ignore non-disease information such as symptoms or conditions without a direct disease name.
    - The response should be strictly between **1 to 5 words**, with no additional information.
    """
)


In [8]:
# Initialize LLM chain
chain = LLMChain(llm=llm, prompt=prompt)

# Function to get disease prediction from LLM
def mistral_predict(text):
    response = chain.invoke({"question": text})
    return response.get("text", "").strip()

# Function to extract disease name from LLM response
def extract_disease(output_text):
    output_text = output_text.strip("**")  # Remove potential markdown formatting
    if output_text.lower() == "no disease detected":
        return "No Disease"
    return output_text



In [9]:
# Function to generate probability outputs for SHAP
def mistral_predict_proba_shap(texts):
    predictions = []
    for text in texts:
        predicted_disease = extract_disease(mistral_predict(text))

        # Assign probabilities
        if predicted_disease == "No Disease":
            prob = [0.5, 0.5]  # Neutral probability if no disease detected
        else:
            prob = [0.8, 0.2]  # Higher confidence if a disease is detected

        predictions.append(prob[0])  # SHAP expects single output probability

    return np.array(predictions)


In [15]:
import shap
import numpy as np

# Define SHAP explainer with a Text masker
text_masker = shap.maskers.Text()
explainer = shap.Explainer(mistral_predict_proba_shap, text_masker)

# Compute SHAP values
sample_texts = [input_text]   # Select 5 texts
shap_values = explainer(sample_texts)

# Visualize SHAP explanations
shap.text_plot(shap_values)  # Use text_plot instead of summary_plot


  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [18:18, 1099.00s/it]             


In [19]:
# sample_texts = input_texts[:5]  # First 5 samples

for text in sample_texts:
    prediction = mistral_predict(text)
    print(f"🔹 Input: {text}\n🔸 LLM Prediction: {prediction}\n")


🔹 Input: Doctor, my mood has been all over the place. Sometimes I feel extremely energetic, like I can do anything, and I don’t need much sleep. My mind races with ideas, and I feel unstoppable. But then I crash and feel hopeless.
🔸 LLM Prediction: **Bipolar Disorder**



In [20]:
# Print SHAP values
for i in range(len(sample_texts)):
    print(f"🔹 Input: {sample_texts[i]}")
    print(f"🔸 SHAP Values: {shap_values[i].values}")  # Check values per token
    print(f"🔹 Feature Importance: {shap_values[i].data}")  # Check tokens


🔹 Input: Doctor, my mood has been all over the place. Sometimes I feel extremely energetic, like I can do anything, and I don’t need much sleep. My mind races with ideas, and I feel unstoppable. But then I crash and feel hopeless.
🔸 SHAP Values: [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
🔹 Feature Importance: ['Doctor, ' 'my ' 'mood ' 'has ' 'been ' 'all ' 'over ' 'the ' 'place. '
 'Sometimes ' 'I ' 'feel ' 'extremely ' 'energetic, ' 'like ' 'I ' 'can '
 'do ' 'anything, ' 'and ' 'I ' 'don’' 't ' 'need ' 'much ' 'sleep. '
 'My ' 'mind ' 'races ' 'with ' 'ideas, ' 'and ' 'I ' 'feel '
 'unstoppable. ' 'But ' 'then ' 'I ' 'crash ' 'and ' 'feel ' 'hopeless']
